In [1]:
"""h5形式のモデルをSavedModelへ変換し、SageMakerのエンドポイントへデプロイする

ref: https://dev.classmethod.jp/articles/vpc-access-sagemaker-notebook/
"""
import boto3
import json
import os 
 
import sagemaker
from sagemaker import get_execution_role
from sagemaker.tensorflow.serving import Model
 
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import model_from_json, load_model
from tensorflow.python.saved_model import builder
from tensorflow.python.saved_model import tag_constants
from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
 
# IAMロール定義取得
role = get_execution_role()

In [3]:
# h5形式のモデル読み込み
model = load_model('model.h5')

# SavedModel形式への変換
input_sig  = 'inputs'
output_sig = 'score'
signature = predict_signature_def(inputs={input_sig: model.input}, outputs={output_sig:model.output})
model_version = '1'
export_path   = './tf_model/mnist/' + model_version
if not os.path.exists(export_path):
    os.mkdir(export_path)

with K.get_session() as sess:
    builder = tf.saved_model.builder.SavedModelBuilder(export_path)
    builder.add_meta_graph_and_variables(sess, [tf.saved_model.tag_constants.SERVING], signature_def_map={'serving_default': signature}, main_op=tf.tables_initializer())
    builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./tf_model/mnist/1/saved_model.pb


In [4]:
# サーバ側の推論用ソースコードを含めた形でSavedModel形式のモデルを圧縮
!cd /home/ec2-user/SageMaker
! tar zcvf model.tar.gz  tf_model/mnist tf_model/code

tf_model/mnist/
tf_model/mnist/.ipynb_checkpoints/
tf_model/mnist/1.org/
tf_model/mnist/1.org/saved_model.pb
tf_model/mnist/1.org/variables/
tf_model/mnist/1.org/variables/variables.index
tf_model/mnist/1.org/variables/variables.data-00000-of-00001
tf_model/mnist/1/
tf_model/mnist/1/saved_model.pb
tf_model/mnist/1/variables/
tf_model/mnist/1/variables/variables.index
tf_model/mnist/1/variables/variables.data-00000-of-00001
tf_model/code/
tf_model/code/inference.py


In [5]:
# S3へモデルデータをアップロード
sagemaker_session  = sagemaker.Session()
model_data = sagemaker_session.upload_data(path='tf_model/model.tar.gz', key_prefix='model')
print("done")

done


In [7]:
# エンドポイントへモデルをデプロイ
tensorflow_serving_mnist_model = Model(model_data=model_data,
                                        role=role,
                                        framework_version='1.15.5')
predictor = tensorflow_serving_mnist_model.deploy(initial_instance_count=1,
                                                    instance_type='ml.t2.medium',
                                                    endpoint_name='mnist-endpoint')

The class sagemaker.tensorflow.serving.Model has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


------!

In [9]:
# 推論の動作確認
import pickle as pkl
img = pkl.load(open("data/mnist.pickle", "rb"))
len(img[0][0]), len(img[0][1])
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, 28, 28)
else:
    x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_train = x_train.astype('float32')
x_train /= 255
client = boto3.client("sagemaker-runtime")
res = client.invoke_endpoint(EndpointName=predictor.endpoint_name,
                            Body=json.dumps({'inputs':[x_train[0].tolist()]}),
                            ContentType='application/json',
                            Accept='application/json')
result = json.loads(res['Body'].read())

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [10]:
result

{'outputs': [[1.31569811e-10,
   4.00890077e-09,
   7.53057269e-11,
   0.0169722475,
   1.24603617e-11,
   0.983027697,
   7.75427367e-09,
   1.49165791e-10,
   4.87038925e-08,
   5.02105415e-08]]}